In [72]:
from sqlalchemy import create_engine


engine = create_engine('mysql+pyodbc://telecom')
%load_ext sql
%sql engine
%config SqlMagic.autopandas = True

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
%sql SHOW tables

# Sample Users
```sql
CREATE TABLE sample_user_201308 (
    # SERV_ID: user_id, MSISDN: cellphone number, CERT_NBR: identity number, CI_BRANCH: branch office
	SELECT SERV_ID, MSISDN, MONTH_NO, CERT_NBR, CI_BRANCH, 
		CASE WHEN
            (
                DETAIL_NAME IS NOT NULL AND 
                DETAIL_NAME!='¸öÈË¿Í»§' AND                # 个人客户
                DETAIL_NAME!='¼ÒÍ¥¿Í»§' AND                # 家庭客戶
                DETAIL_NAME!='Å©´å¹«ÖÚ£¨¼ÒÍ¥¼°¸öÈË£©' AND  # 农村公众（家庭及个人）
                DETAIL_NAME!='Å©´å¿Í»§' AND                # 农村客户
                DETAIL_NAME!='³ÇÊÐ¸öÈË¿Í»§' AND            # 城市个人客户
                DETAIL_NAME!='³ÇÊÐ¼ÒÍ¥¿Í»§'                # 城市家庭客户
             )
			 OR 
             (
                 (CI_IVPN_FLAG=1 OR VPN_FLAG=1) AND 
                 (PL_IVPN_CAT IS NULL OR PL_IVPN_CAT='ÐéÄâÍø' OR PL_IVPN_CAT='¼¯ÍÅvÍø')  # 虚拟网, 集团v网
             )
			OR 
            (
                MB_ENPR_FLAG_M1=1 OR   # individual works for government
                IS_BUSINESS='1' OR     # wheter pay the fee
                RED_MARK=1 OR          # 省市领导级别的人
                PL_BUSINESS_FLAG=1 OR  # business user
                IS_ZQJN=1 OR           # whether the gorvernment & enterprise cluster	
                IS_ZQHY=1              # whether the government & enterprise industry
            )
		THEN 1 ELSE 0 END AS EMPLOY_FLAG, 
		CASE WHEN 
            DETAIL_NAME='¸ßµÈºÍÖ°ÒµÔºÐ£' OR  # 高等和职业院校
            PL_IVPN_CAT='Ð£Ô°VÍø' OR         # 校园V网
            PL_CAMPUS_FLAG=1 OR 
            IS_SCHOOL='1'
		THEN 1 ELSE 0 END AS STUDENT_FLAG, 
		CASE WHEN 
            (
                DETAIL_NAME='Å©´å¹«ÖÚ£¨¼ÒÍ¥¼°¸öÈË£©' OR  # 农村公众（家庭及个人）
                DETAIL_NAME='Å©´å¿Í»§'                   # 农村客户
            ) 
            OR 
            PL_IVPN_CAT='ÏçÇévÍø' OR  # 乡情v网
            BSS_ORG_ZJ_FLAG='1'
		THEN 1 ELSE 0 END AS RURAL_FLAG, 
		HS_CDMA_BRAND AS TERMINAL_BRAND, 
        HS_CDMA_MODEL AS TERMINAL_MODEL, 
		CASE IS_INTELLIGENT WHEN 'ÊÇ' THEN 1 ELSE 0 END AS TERMINAL_SMART_FLAG,  # 是
		TERMINAL_TYPE,            # 3G, 1x, None
		CASE HS_CDMA_LAYER
			WHEN '³¬µÍ¶Ë' THEN 1  # 超低端
			WHEN 'µÍ¶Ë' THEN 2    # 低端
			WHEN 'ÖÐ¶Ë' THEN 3    # 中端
			WHEN '¸ß¶Ë' THEN 4    # 高端
			WHEN '³¬¸ß¶Ë' THEN 5  # 超高端
        END AS TERMINAL_LEVEL, 
		HS_CDMA_TER_PRICE AS TERMINAL_PRICE, 
		HS_CDMA_CT_DATE, 
		PD_EVDO_FLAG_M1 AS PD_EVDO_FLAG, 
        PD_1X_FLAG_M1 AS PD_1X_FLAG, 
		MB_ARPU_CDMA_M1 AS ARPU, 
        MB_ARPU_CDMA_ALL_M1 AS ARPU_ALL, 
		PL_CONTRACT_FLAG AS CONTRACT_FLAG, 
        PL_EXPIRE_MONTH AS CONTRACT_EXPIRE_MONTH, 
		VO_CDMA_MOUOUT_LOCAL_M1 AS VO_MOU_LOCAL, 
        VO_CDMA_MOU_DIST_M1 AS VO_MOU_DIST
	FROM tb_asz_cdma_0838_201308
	WHERE 
        CERT_NBRR LIKE '[1-8][0-7]-[0-7][0-9]-[0-9][0-9]-[12][09][0-9][0-9]-[01][0-9]-[0-3][0-9]-[01]' AND     
        SUBSTR(CERT_NBR,10,4)>='1920' AND 
        CONVERT(SUBSTR(CERT_NBR,10,4),UNSIGNED)<=CONVERT(SUBSTR(MONTH_NO,1,4),UNSIGNED)-16 AND 
        SUBSTR(CERT_NBR,15,2)>='01' AND 
        SUBSTR(CERT_NBR,15,2)<='12' AND 
        SUBSTR(CERT_NBR,18,2)>='01' AND 
        SUBSTR(CERT_NBR,18,2)<='31'AND 
        PL_BB_FLAG=0 AND                       # 是否無線
        IS_WX_FLAG=0 AND                       # 是否無線
        IS_CARDPHONE='·ñ' AND                  # 非卡式機
        HS_CDMA_MODEL NOT LIKE '%(¹Ì¶¨Ì¨)'AND  # 非%(固定台)
        IS_INTELLIGENT IS NOT NULL AND 
        HS_CDMA_LAYER IS NOT NULL AND 
        PAYMENT_FLAG=1 AND 
        PD_CDMA_STATUS!='ÓÃ»§ÒªÇóÍ£»ú' AND     # 非用户要求停机
        PD_CDMA_STATUS!='Î´Öª×´Ì¬'             # 非未知狀態
);
CREATE INDEX individual_sampleuser_201308 ON sample_user_201308(SERV_ID, MSISDN);
```

In [51]:
%%sql

SELECT DISTINCT HS_CDMA_LAYER
FROM tb_asz_cdma_0838_201308

Running query in 'mysql+pyodbc://telecom'

6 rows affected.

,HS_CDMA_LAYER
0,超高端
1,中端
2,None
3,低端
4,超低端
5,高端


In [88]:
print("DETAIL_NAME")
print("Å©´å¹«ÖÚ£¨¼ÒÍ¥¼°¸öÈË£©".encode('iso-8859-1').decode('gbk'))
print("Å©´å¿Í»§".encode('iso-8859-1').decode('gbk'))
print("¸ßµÈºÍÖ°ÒµÔºÐ£".encode('iso-8859-1').decode('gbk'))
print("¸öÈË¿Í»§".encode('iso-8859-1').decode('gbk'))
print("¼ÒÍ¥¿Í»§".encode('iso-8859-1').decode('gbk'))
print("³ÇÊÐ¸öÈË¿Í»§".encode('iso-8859-1').decode('gbk'))
print("³ÇÊÐ¼ÒÍ¥¿Í»§".encode('iso-8859-1').decode('gbk'))
print()


print("PL_IVPN_CAT")
print("ÏçÇévÍø".encode('iso-8859-1').decode('gbk'))
print("Ð£Ô°VÍø".encode('iso-8859-1').decode('gbk'))
print("ÐéÄâÍø".encode('iso-8859-1').decode('gbk'))
print("¼¯ÍÅvÍø".encode('iso-8859-1').decode('gbk'))
print()


print("TERMINAL_LEVEL")
print("µÍ¶Ë".encode('iso-8859-1').decode('gbk'))
print("ÖÐ¶Ë".encode('iso-8859-1').decode('gbk'))
print("¸ß¶Ë".encode('iso-8859-1').decode('gbk'))
print()

print("terminal smart flag")
print("ÊÇ".encode('iso-8859-1').decode('gbk'))
print()

print("IS_CARD_PHONE")
print("·ñ".encode('iso-8859-1').decode('gbk'))         
print()


print("HS_CDMA_MODEL")
print("%(¹Ì¶¨Ì¨)".encode('iso-8859-1').decode('gbk'))
print()


print("PD_CDMA_STATUS")
print("ÓÃ»§ÒªÇóÍ£»ú".encode('iso-8859-1').decode('gbk'))
print("Î´Öª×´Ì¬".encode('iso-8859-1').decode('gbk'))

DETAIL_NAME
农村公众（家庭及个人）
农村客户
高等和职业院校
个人客户
家庭客户
城市个人客户
城市家庭客户

PL_IVPN_CAT
乡情v网
校园V网
虚拟网
集团v网

TERMINAL_LEVEL
低端
中端
高端

terminal smart flag
是

IS_CARD_PHONE
否

HS_CDMA_MODEL
%(固定台)

PD_CDMA_STATUS
用户要求停机
未知状态


## inspect the meaning of the wireless

In [145]:
df = %sql SELECT DISTINCT HS_CDMA_MODEL FROM tb_asz_cdma_0838_201308

Running query in 'mysql+pyodbc://telecom'

1479 rows affected.

In [144]:
check = df.HS_CDMA_MODEL.str.contains("台").apply(lambda x: x if x is not None else False)
df.HS_CDMA_MODEL[check]

381    中兴-WP228(固定台)
Name: HS_CDMA_MODEL, dtype: object

In [155]:
%%sql

SELECT
    DETAIL_NAME,
    COUNT(*)
FROM tb_asz_cdma_0838_201308
WHERE PL_BB_FLAG=1 OR IS_WX_FLAG=1
GROUP BY DETAIL_NAME
ORDER BY COUNT(*) DESC

Running query in 'mysql+pyodbc://telecom'

20 rows affected.

,DETAIL_NAME,COUNT(*)
0,大企业【不含农村】,3518
1,个人客户,3355
2,家庭客户,1975
3,市县级党政军,1473
4,金融,1421
5,农村公众（家庭及个人）,1167
6,农村政企【不含大企业】,620
7,医院,378
8,None,261
9,中小企业,130


# Sample sight
```sql
#CREATE INDEX individual_servacct_201308 ON serv_acct_item_0838_201308(SERV_ID, ACC_NBR);
CREATE TABLE sample_sight_201308 (
	SELECT 
        A.START_TIME AS DATE_TIME,
        A.ACC_NBR, A.MONTH_NO, A.LAC, A.CELL_ID, A.ETL_TYPE_ID, A.CALLING_NBR, A.CALLED_NBR,
        B.SERV_ID, 
        C.LON, C.LAT, C.X, C.Y, C.HOT_FLAG, C.HOT_MAIN_FLAG, C.HOT_OTHER_FLAG, 
	FROM serv_acct_item_0838_201308 A
	LEFT JOIN sample_user_201308 B ON A.SERV_ID=B.SERV_ID AND A.ACC_NBR=B.MSISDN
	LEFT JOIN tower_hot_201308 C ON A.CELL_ID=C.CELL_16
	WHERE A.LAC!='-1' AND A.LAC!='0'
);
CREATE INDEX user_sample_201308 ON sample_sight_201308(SERV_ID);
DELETE FROM sample_sight_201308
WHERE ISNULL(SERV_ID);
CREATE INDEX individual_sample_201308 ON sample_sight_201308(SERV_ID, ACC_NBR);
```